# Задания

In [19]:
# Обновляем библиотеки pandas, gdown, reverse_geocode, pycountry plotly до последних версий.
!pip install --upgrade pandas gdown reverse_geocoder pycountry plotly

# Импортируем библиотеки после их обновления.
import numpy as np
import pandas as pd
import gdown
import reverse_geocoder as rg
import pycountry
from sklearn.cluster import MeanShift
import warnings                                                           # Импорт модуля для работы с предупреждениями
from sklearn.exceptions import DataConversionWarning                      # Импорт класса DataConversionWarning для обработки предупреждений
warnings.filterwarnings(action='ignore', category=FutureWarning)          # Игнорирование предупреждений типа FutureWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)  # Игнорирование предупреждений типа DataConversionWarning
from scipy.spatial.distance import euclidean                              # Импортируем функцию euclidean для вычисления евклидова расстояния
import plotly.express as px

# Загружаем файлы checkins.dat и offices.csv с Google Drive, используя их ID
# Указываем параметр quiet=False, чтобы отображать информацию о процессе скачивания
gdown.download("https://drive.google.com/uc?id=1BVXlbGoIk64wdwORAKG8bj5NR1uohD5W", "checkins.dat", quiet=False)
gdown.download("https://drive.google.com/uc?id=1YSeuhhjqt__AcPNPOf4dMrssWyvuk5Kl", "offices.csv", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1BVXlbGoIk64wdwORAKG8bj5NR1uohD5W
To: /content/checkins.dat
100%|██████████| 94.0M/94.0M [00:00<00:00, 161MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YSeuhhjqt__AcPNPOf4dMrssWyvuk5Kl
To: /content/offices.csv
100%|██████████| 281/281 [00:00<00:00, 1.27MB/s]


'offices.csv'

## Задание 1

Загрузи данные о посещениях заведений. В них содержится информация о регистрации пользователя в заведениях
и геолокация этих заведений. Очисти данные от записей с пропусками. \
Выведи количество записей после очистки.

In [20]:
# Определяем параметры чтения CSV-файла.
csv_params = {
    'sep': '|',                # Указываем разделитель - вертикальная черта
    'header': 0,               # Первая строка файла - заголовок
    'skipinitialspace': True,  # Пропускаем начальные пробелы в значениях
    'names': ['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at']  # Задаем имена столбцов как в файле checkins.dat
}

# Читаем данные из checkins.dat с указанными параметрами и отключаем предупреждение о смешанных типах данных
data = pd.read_csv('checkins.dat', **csv_params, low_memory=False)

# Удаляем строки с отсутствующими значениями (NaN) из DataFrame
data.dropna(inplace=True)

# Выводим количество записей после очистки
data_len = len(data)

# Выводим результаты
print(f"Количество записей после очистки: {data_len}")

Количество записей после очистки: 396634


## Задание 2

Эти данные содержат информацию о заведениях со всего мира. С помощью геолокаций и библиотеки [Reverse Geocoder](https://github.com/thampiman/reverse-geocoder),
узнай страну для каждой геопозиции. \
Узнай **название** второй страны по количеству записей.

In [21]:
# Создаем кортежи из широты и долготы
location_tuples = list(zip(data['latitude'], data['longitude']))

# Используем Reverse Geocoder для определения страны
results = rg.search(location_tuples)

# Создаем новый столбец 'country' и заполняем его кодами стран
data['country'] = [result['cc'] for result in results]

# Группируем данные по столбцу 'country' и подсчитываем количество записей для каждой страны
country_counts = data.groupby('country').size()

# Подсчет количества записей по странам
country_counts = data['country'].value_counts()

# Получаем код второй по популярности страны
second_country_code = country_counts.index[1]

# Используем pycountry для получения информации о стране
second_country_info = pycountry.countries.get(alpha_2=second_country_code)

# Получаем название второй по популярности страны
second_country_name = second_country_info.name

# Выводим название второй по популярности страны
print("Название второй страны по количеству записей:", second_country_name)

Название второй страны по количеству записей: Indonesia


## Задание 3

Нас будут интересовать только американские локации. Очисти данные от локаций, находящихся в других странах.
Также, чтобы сократить количество, геолокаций оставь в выборке только 50 самых часто встречаемых заведений (venue). \
Выведи количество локаций, оставшихся после этих очисток.

In [22]:
# Фильтруем данные, оставляя только американские локации
data_us = data[data['country'] == 'US']

# Получаем список 50 самых часто встречаемых заведений (venue_id)
top_50_venues = data_us['venue_id'].value_counts().head(50).index.tolist()

# Фильтруем исходный датафрейм по стране и venue_id из списка top_50_venues
filtered_data = data[(data['country'] == 'US') & (data['venue_id'].isin(top_50_venues))]

# Выводим количество оставшихся локаций
print("Количество локаций, оставшихся после очисток:", len(filtered_data))

Количество локаций, оставшихся после очисток: 162099


## Задание 4

Перейдем к задаче кластеризации. Воспользуйся алгоритмом [Mean Shift](https://scikit-learn.org/stable/modules/clustering.html#mean-shift)
для кластеризации локаций. Параметрами укажи `MeanShift(bandwidth=0.1, bin_seeding=True)`.

    `bandwidth=0.1` - это ширина ядра кластеризации. Для средних широт США - это порядка 5-10 км.
    `bin_seeding=True` - для ускорения работы алгоритма.
    
Выведи количество кластеров, которые у тебя получились в результате кластеризации.

In [23]:
# Создаем объект MeanShift для кластеризации с указанными параметрами
mean_shift = MeanShift(bandwidth=0.1, bin_seeding=True)

# Применяем алгоритм MeanShift к координатам широты и долготы из отфильтрованных данных
clusters = mean_shift.fit_predict(filtered_data[['latitude', 'longitude']])

# Определяем количество уникальных кластеров, используя функцию set(), и сохраняем его в переменную "num_clusters"
num_clusters = len(set(clusters))

# Выводим количество кластеров получившихся в результате кластеризации
print(f"Количество кластеров после кластеризации: {num_clusters}")

Количество кластеров после кластеризации: 2846


## Задание 5

Центры полученных кластеров - это потенциальные места установки банеров компании. Теперь хотелось бы найти те центры кластеров,
которые наиболее близко находятся к офисам продаж компании. \
Загрузи [данные по координатам офисов компании](datasets/offices.csv). Для каждого офиса найди 5 самых ближайших к нему центров кластеров.
(Пренебрежем тем, что Земля круглая и рассчитаем Евклидово расстояние).
У компании 11 офисов, значит у нас должно получится 55 мест установки баннеров. \
Выведи координаты установки баннера, который ближе всего находится к офису компании.

In [24]:
from scipy.spatial.distance import cdist

In [25]:
# Загружаем данные о координатах офисов компании
df_offices = pd.read_csv("offices.csv")

# Получаем центры кластеров после кластеризации
cluster_centers = mean_shift.cluster_centers_

# Создаем пустой список для хранения результатов
banner_locations = []

# Для каждого офиса компании
for index, office in df_offices.iterrows():
    office_coords = (office['latitude'], office['longitude'])

    # Рассчитываем расстояния между офисом и всеми центрами кластеров (Евклидово расстояние)
    distances = cdist([office_coords], cluster_centers, metric='euclidean')[0]

    # Находим индексы 5 ближайших центров
    nearest_clusters_indices = np.argpartition(distances, 5)[:5]

    # Добавляем информацию о ближайших баннерах для текущего офиса
    for idx in nearest_clusters_indices:
        banner_coords = cluster_centers[idx]
        distance = distances[idx]
        banner_locations.append({
            'office_coords': office_coords,
            'nearest_banner_coords': banner_coords,
            'distance': distance
        })

# Сортируем список по расстоянию и выбираем ближайший баннер для каждого офиса
banner_locations.sort(key=lambda x: x['distance'])
nearest_banner_info = banner_locations[0]

# Вычисляем количество мест для установки баннеров, координаты офиса компании и координаты ближайшего баннера
num_banner_locations = len(banner_locations)
office_coords = nearest_banner_info['office_coords']
nearest_banner_coords = nearest_banner_info['nearest_banner_coords']

# Выводим результаты
print(f"Количество мест для установки баннеров: {num_banner_locations}")
print(f"Координаты офиса компании: {office_coords}")
print(f"Координаты ближайшего баннера: {nearest_banner_coords}")

Количество мест для установки баннеров: 55
Координаты офиса компании: (32.786669, -79.928605)
Координаты ближайшего баннера: [ 32.78531778 -79.92474241]


## Задание 6

С помощью функции [scatter_mapbox](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html)
отметь точки установки баннеров. У тебя должна получится такая картинка.
Цветом точки укажи к какому офису будет относиться этот баннер.
>Цвет легенды может отличаться от референса

<center><img src="../misc/images/task_6.png" width="800" height="800"/> <center/>

In [33]:
banner_locations

[{'office_coords': (32.786669, -79.928605),
  'nearest_banner_coords': array([ 32.78531778, -79.92474241]),
  'distance': 0.004092113620846756},
 {'office_coords': (29.307715, -94.797707),
  'nearest_banner_coords': array([ 29.3013479, -94.7976958]),
  'distance': 0.006367109850632917},
 {'office_coords': (30.686856, -88.038649),
  'nearest_banner_coords': array([ 30.6943566, -88.0430541]),
  'distance': 0.008698500236826535},
 {'office_coords': (32.717588, -117.174758),
  'nearest_banner_coords': array([  32.71596285, -117.15819697]),
  'distance': 0.016640581112594262},
 {'office_coords': (27.945382, -82.4439174),
  'nearest_banner_coords': array([ 27.94946081, -82.46497056]),
  'distance': 0.02144462830699618},
 {'office_coords': (39.2640889, -76.5892116),
  'nearest_banner_coords': array([ 39.28729393, -76.61349328]),
  'distance': 0.033586803942988186},
 {'office_coords': (25.8527884, -80.1913388),
  'nearest_banner_coords': array([ 25.7869859 , -80.21855932]),
  'distance': 0.071

In [34]:
df_banner_locations = pd.DataFrame(banner_locations)
df_banner_locations['latitude'] = df_banner_locations['office_coords'].apply(lambda x: x[0])
df_banner_locations['longitude'] = df_banner_locations['office_coords'].apply(lambda x: x[1])
df_banner_locations['ban_latitude'] = df_banner_locations['nearest_banner_coords'].apply(lambda x: x[0])
df_banner_locations['ban_longitude'] = df_banner_locations['nearest_banner_coords'].apply(lambda x: x[1])
df_banner_locations

,office_coords,nearest_banner_coords,distance,latitude,longitude,ban_latitude,ban_longitude
0,"(32.786669, -79.928605)","[32.78531777579914, -79.92474241187222]",0.004092,32.786669,-79.928605,32.785318,-79.924742
1,"(29.307715, -94.797707)","[29.3013479, -94.79769579999999]",0.006367,29.307715,-94.797707,29.301348,-94.797696
2,"(30.686856, -88.038649)","[30.694356599999995, -88.04305410000002]",0.008699,30.686856,-88.038649,30.694357,-88.043054
3,"(32.717588, -117.174758)","[32.715962853674256, -117.15819696624045]",0.016641,32.717588,-117.174758,32.715963,-117.158197
4,"(27.945382, -82.4439174)","[27.949460812240474, -82.46497055591384]",0.021445,27.945382,-82.443917,27.949461,-82.464971
5,"(39.2640889, -76.5892116)","[39.28729392569308, -76.6134932840785]",0.033587,39.264089,-76.589212,39.287294,-76.613493
6,"(25.8527884, -80.1913388)","[25.78698590071344, -80.21855931584689]",0.071210,25.852788,-80.191339,25.786986,-80.218559
7,"(33.751277, -118.18874)","[33.80980562408899, -118.14497105040418]",0.073084,33.751277,-118.188740,33.809806,-118.144971
8,"(34.272249, -118.46842)","[34.18768865305686, -118.44880518275103]",0.086805,34.272249,-118.468420,34.187689,-118.448805
9,"(30.4068247, -81.5788387)","[30.33243195694452, -81.65492676666666]",0.106413,30.406825,-81.578839,30.332432,-81.654927


In [35]:
# Загружаем данные о координатах офисов компании
df_offices = pd.read_csv("offices.csv")

# Порядок в легенде как на картинке
order = [9, 4, 5, 3, 6, 10, 7, 2, 1, 8, 0]
office = df_offices.reindex(order)

In [36]:
df_closest_office=office.merge(df_banner_locations, on='latitude')
df_last=df_closest_office.rename(columns={'Unnamed: 0':'index_office'})
df_last

,index_office,latitude,longitude_x,office_coords,nearest_banner_coords,distance,longitude_y,ban_latitude,ban_longitude
0,9,32.786669,-79.928605,"(32.786669, -79.928605)","[32.78531777579914, -79.92474241187222]",0.004092,-79.928605,32.785318,-79.924742
1,9,32.786669,-79.928605,"(32.786669, -79.928605)","[32.67952435, -79.98575185]",0.121432,-79.928605,32.679524,-79.985752
2,9,32.786669,-79.928605,"(32.786669, -79.928605)","[32.9810059, -80.0325867]",0.220406,-79.928605,32.981006,-80.032587
3,9,32.786669,-79.928605,"(32.786669, -79.928605)","[32.60823729999999, -80.08481659999998]",0.237150,-79.928605,32.608237,-80.084817
4,9,32.786669,-79.928605,"(32.786669, -79.928605)","[33.01850389999999, -80.1756481]",0.338789,-79.928605,33.018504,-80.175648
5,4,29.307715,-94.797707,"(29.307715, -94.797707)","[29.3013479, -94.79769579999999]",0.006367,-94.797707,29.301348,-94.797696
6,4,29.307715,-94.797707,"(29.307715, -94.797707)","[29.420566, -94.92420136000001]",0.169517,-94.797707,29.420566,-94.924201
7,4,29.307715,-94.797707,"(29.307715, -94.797707)","[29.53663335, -95.08341670833329]",0.366106,-94.797707,29.536633,-95.083417
8,4,29.307715,-94.797707,"(29.307715, -94.797707)","[29.6890241, -95.00539099999999]",0.434200,-94.797707,29.689024,-95.005391
9,4,29.307715,-94.797707,"(29.307715, -94.797707)","[29.4238472, -95.2441009]",0.461253,-94.797707,29.423847,-95.244101


In [37]:
df_last.dtypes

index_office               int64
latitude                 float64
longitude_x              float64
office_coords             object
nearest_banner_coords     object
distance                 float64
longitude_y              float64
ban_latitude             float64
ban_longitude            float64
dtype: object

In [38]:
df_last['index_office']=df_last['index_office'].astype(str)

In [39]:
fig = px.scatter_mapbox(df_last, lat="ban_latitude", lon="ban_longitude",  color="index_office", mapbox_style='open-street-map',hover_name='index_office', labels='index_office', width=900, height=1000, zoom=3 )

fig.show()